# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
# nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# scikit-learn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
# pickle 
import pickle

In [11]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM DisasterResponse', engine)
X = df.message
#Y = df.drop(columns=['id','message','original','genre'])
Y = df[['medical_help','military']]
category_names = Y.columns

In [12]:
category_names

Index(['medical_help', 'military'], dtype='object')

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok.lower().strip())
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf',MultiOutputClassifier(RandomForestClassifier(verbose=1,n_jobs=-1,random_state=42)))
                         ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.6s finished


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
# predict on test data
y_pred = pipeline.predict(X_test)
for idx, col in enumerate(category_names):
    print('For category {}:'.format(col))
    print(classification_report(y_test[col], y_pred[:,idx]))

For category medical_help:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6026
           1       0.63      0.03      0.06       528

    accuracy                           0.92      6554
   macro avg       0.78      0.52      0.51      6554
weighted avg       0.90      0.92      0.89      6554

For category military:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6338
           1       0.73      0.04      0.07       216

    accuracy                           0.97      6554
   macro avg       0.85      0.52      0.53      6554
weighted avg       0.96      0.97      0.95      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# specify parameters for grid search
parameters = {'clf__estimator__max_features' : [None, 'sqrt', 'log2'],
              'clf__estimator__min_samples_leaf': [1, 2, 5],
            'clf__estimator__n_estimators': [100, 200, 500]
             }
# create grid search object
pipeline_cv = GridSearchCV(pipeline, parameters)
pipeline_cv.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print(pipeline.get_params)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
# predict on test data
y_pred = pipeline_cv.predict(X_test)
for idx,col in enumerate(category_names):
    print('For category {}:'.format(col))
    print(classification_report(y_test[col], y_pred[:,idx]))

For category medical_help:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6026
           1       0.63      0.03      0.06       528

    accuracy                           0.92      6554
   macro avg       0.78      0.52      0.51      6554
weighted avg       0.90      0.92      0.89      6554

For category military:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6338
           1       0.73      0.04      0.07       216

    accuracy                           0.97      6554
   macro avg       0.85      0.52      0.53      6554
weighted avg       0.96      0.97      0.95      6554



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# tbd

### 9. Export your model as a pickle file

In [22]:
model_filepath = './data/model'
outfile = open(model_filepath,'wb')
pickle.dump(pipeline, outfile)
outfile.close()

In [23]:
infile = open(model_filepath,'rb')
clf2 = pickle.load(infile)
infile.close()
# predict on test data
y_pred = clf2.predict(X_test)
for idx,col in enumerate(y_test):
    print('For category {}:'.format(col))
    print(classification_report(y_test[col], y_pred[:,idx]))

For category medical_help:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6026
           1       0.63      0.03      0.06       528

    accuracy                           0.92      6554
   macro avg       0.78      0.52      0.51      6554
weighted avg       0.90      0.92      0.89      6554

For category military:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6338
           1       0.73      0.04      0.07       216

    accuracy                           0.97      6554
   macro avg       0.85      0.52      0.53      6554
weighted avg       0.96      0.97      0.95      6554



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.